## Basic Numpy Math

Key points:
 - Addition
 - Multiplication of signle row or column vectors

In [2]:
import numpy as np

## Creating NumPy Arrys

In [37]:
# access shape
def print_shape(a, name):
    if len(a.shape) == 1:
        print(f"{name}: flat/1D vector: {a.shape}, ndim:{a.ndim}")
    elif len(a.shape) == 2:
        r = a.shape[0]
        c = a.shape[1]
        print(f"{name}: 2D matrix {a.shape}: {r} rows, {c} columns, ndim: {a.ndim}")
    else:
        print(f"{name}: High dimensional array: {a.shape}, ndim: {a.ndim}")

In [38]:
a1 = np.array([1, 2, 3])  # 1D array, not strictly a row or column
a2 = np.array([[4], [5], [6]]) # 3 rows, 1 column
a3 = np.array([[-4], [-5], [-6]]) # 3 rows, 1 column
a4 = np.array([[1,2,3]]) # 1 row, 3 columns

for i, a in enumerate([a1, a2, a3, a4]):
    print_shape(a, f"a{i}")
    print(a)

a0: flat/1D vector: (3,), ndim:1
[1 2 3]
a1: 2D matrix (3, 1): 3 rows, 1 columns, ndim: 2
[[4]
 [5]
 [6]]
a2: 2D matrix (3, 1): 3 rows, 1 columns, ndim: 2
[[-4]
 [-5]
 [-6]]
a3: 2D matrix (1, 3): 1 rows, 3 columns, ndim: 2
[[1 2 3]]


__Notes:__:

1. a1 (3,)	 is a flat vector, used most commonly in NumPy.
2. a1 (1, 3) is a 2D row vector, which is useful when performing matrix operations (e.g., dot product).

__Best Practice:__
1. Use (3,) if you’re working with plain vectors or broadcasting.
2. Use (1, 3) if you need to preserve row/column shape in linear algebra.

__reshape():__
1. It **never modifies in-place** — you must reassign: a = a.reshape(...) if you want to change a.
2. usually returns a view, so it’s memory-efficient.
3. If not possible (due to layout), it returns a copy.

In [13]:
a1_reshaped = a1.reshape(-1, 1)
print(f"Shape of a1_reshaped: {a1_reshaped.shape}, ndim: {a1_reshaped.ndim}")
a1_reshaped[1,0] = 100
print(a1_reshaped)
a1_reshaped2 = a1.reshape(1, -1)
print(
    f"Shape of a1_reshaped2: {a1_reshaped2.shape}, ndim: {a1_reshaped2.ndim}")
print(a1_reshaped2)

Shape of a1_reshaped: (3, 1), ndim: 2
[[  1]
 [100]
 [  3]]
Shape of a1_reshaped2: (1, 3), ndim: 2
[[  1 100   3]]


In [15]:
print(a1)
print(f"a1: {a1.shape}, ndim: {a1.ndim}")
print(f"a1_reshaped: {a1_reshaped.shape}, ndim: {a1_reshaped.ndim}")
print(f"a1_reshaped2: {a1_reshaped2.shape}, ndim: {a1_reshaped2.ndim}")


[  1 100   3]
a1: (3,), ndim: 1
a1_reshaped: (3, 1), ndim: 2
a1_reshaped2: (1, 3), ndim: 2


In [27]:
print_shape(a1, "a1")
print_shape(a2, "a2")
print_shape(a3, "a3")
print_shape(a1_reshaped, "a1_reshaped")
print_shape(a1_reshaped2, "a1_reshaped2")


a1: Shape is a flat/1D vector: (3,), ndim:1
a2: Shape is a flat/1D vector: (3,), ndim:1
a3: Shape is a flat/1D vector: (3,), ndim:1
a1_reshaped: Shape is a flat/1D vector: (3,), ndim:1
a1_reshaped2: Shape is a flat/1D vector: (3,), ndim:1


In [28]:
a2_ravel = a2.ravel()
print_shape(a2_ravel, "a2_ravel")

a2_ravel: Shape is a flat/1D vector: (3,), ndim:1


## Broadcasting

See [NumPy documentatin](https://numpy.org/doc/stable/user/basics.broadcasting.html).
 - Move Right to left
 - Dimensions are compation when: they are equal or either one is 1

A set of arrays is called “broadcastable” to the same shape if the above rules produce a valid result.



## Axes

Use it on NumPy 
axis = 0, along rows (Add vertically)
axis = 1, along columns

In [40]:
a1, a2

(array([1, 2, 3]),
 array([[4],
        [5],
        [6]]))

In [46]:
# a1: (3,), a2: (3,1)
print_shape(a1, "a1")
print_shape(a2, "a2")
a1_sum_a2 = a1 + a2

a1: flat/1D vector: (3,), ndim:1
a2: 2D matrix (3, 1): 3 rows, 1 columns, ndim: 2


In [44]:
A = np.array([[1,2, 3]])
B = np.array([10, 20, 30])
print_shape(A, "A")
print_shape(B, "B")
C = A + B
print_shape(C, "C")
print(C)

A: 2D matrix (1, 3): 1 rows, 3 columns, ndim: 2
B: flat/1D vector: (3,), ndim:1
C: 2D matrix (1, 3): 1 rows, 3 columns, ndim: 2
[[11 22 33]]


In [53]:
C = np.arange(12).reshape((3,4))
print(C)
col_sum = np.sum(C, axis=0)
print(col_sum)
print_shape(col_sum, "col_sum")
row_sum = np.sum(C, axis=1)
print(row_sum)
print_shape(row_sum, "row_shape")

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]
[12 15 18 21]
col_sum: flat/1D vector: (4,), ndim:1
[ 6 22 38]
row_shape: flat/1D vector: (3,), ndim:1
